In [ ]:
words = open('./names.txt', 'r').read().splitlines()

In [ ]:
words[:10]

In [ ]:
from collections import defaultdict
count = defaultdict(int)
for w in words:
    chs = ['<S>'] + list(w) + ['<E>'] 
    for c1, c2 in zip(chs, chs[1:]):
        bigram = (c1, c2)
        count[bigram] += 1

In [ ]:
count

In [ ]:
sorted(count.items(), key = lambda kv : kv[1], reverse=True)

In [ ]:
import torch

In [ ]:
chars = sorted(list(set(''.join(words))))
len(chars)

In [ ]:
stoi = {s : i for i, s in enumerate(chars)}
stoi['.'] = 26
itos = {i : s for s, i in stoi.items()}
itos

In [ ]:
N = torch.zeros((27, 27), dtype=torch.int32)

In [ ]:
from collections import defaultdict
count = defaultdict(int)
for w in words:
    chs = ['.'] + list(w) + ['.'] 
    for c1, c2 in zip(chs, chs[1:]):
        N[stoi[c1], stoi[c2]] += 1

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(16,16))
plt.imshow(N, cmap='Blues')
for i in range(27):
    for j in range(27):
        chstr = itos[i] + itos[j]
        plt.text(j, i, chstr, ha="center", va="bottom", color='gray')
        plt.text(j, i, N[i, j].item(), ha="center", va="top", color='gray')
plt.axis('off');

In [ ]:
N.sum(dim = 1)

In [ ]:
N.sum(dim = 1).shape

In [ ]:
P = (N + 1).float() 
P /= N.sum(dim = 1, keepdim=True)
P.shape

In [ ]:
P[0].sum()

In [ ]:
ix = 0
for _ in range(20):
    out = []
    while True:
        p = P[ix]   
        ix = torch.multinomial(p, 1, replacement=True).item()
        out.append(itos[ix])
        if ix == 26:
            break
    print(''.join(out))

a * b * c
Likelihood Function:
    L(θ)=P(X1​,X2​,...,Xn​∣θ)=i= prod(​P(Xi​∣θ))


In [ ]:
log_likelihood = 0
n = 0
for w in ["andrekx"]:
    chs = ['.'] + list(w) + ['.']
    for c1, c2 in zip(chs, chs[1:]):
        index_1 = stoi[c1]
        index_2 = stoi[c2]  
        prob = P[index_1][index_2]
        log_prob = torch.log(torch.Tensor(prob))
        log_likelihood += log_prob
        n += 1
        print(f'{c1 + c2} with index {index_1, index_2}, {prob:.4f} {log_prob:.4f}')
print(log_likelihood)
nlh = -log_likelihood
print(nlh)
print(nlh / n) # the lower the better

In [ ]:
# tao training set cua cac bigram
xs, ys = [], []
for w in words[:1]:
    chs = ['.'] + list(w) + ['.'] 
    for c1, c2 in zip(chs, chs[1:]):
        print(c1 + c2)
        index_1 = stoi[c1]
        index_2 = stoi[c2]
        xs.append(index_1)
        ys.append(index_2)
        
xs = torch.tensor(xs)
ys = torch.tensor(ys)
display(xs)
display(ys)
print(itos[xs[0].item()] + itos[ys[1].item()])

In [ ]:
import torch.nn.functional as f

In [ ]:
xs_encoded = f.one_hot(xs, num_classes=27).float()

In [ ]:
xs_encoded.dtype
xs_encoded.shape

In [ ]:
plt.imshow(xs_encoded)

In [ ]:
W = torch.randn((27, 27))
xs_encoded @ W

In [ ]:
W

In [ ]:
logits = xs_encoded @ W
logits

In [ ]:
probs = f.softmax(logits)
probs

In [ ]:
probs.shape

In [ ]:
display(xs)
display(ys)

In [ ]:
W = torch.randn((27, 27), requires_grad = True)

In [ ]:
logits = xs_encoded @ W
probs = f.softmax(logits, dim = 1)
for i in range(len(xs)):
    x = xs[i].item()
    y = ys[i].item()
    bi_gram = itos[x] + itos[y]
    print('The word is ', bi_gram)
    print('The probs is ', probs[i].max())
    print('The predicted word is ', itos[torch.argmax(probs[i]).item()])
    print('Prob for the correct word is', probs[i][y])
    print("=" * 10)
    

In [ ]:
probs[0, 5], probs[1, 13], probs[2, 13]

In [ ]:
# Negative likelihood
-probs[torch.arange(5), ys].log().mean()
# we want all this to one

In [ ]:
display(xs)
display(ys)

In [ ]:
# forward pass
xs_encoded = f.one_hot(xs).float()
logits = xs_encoded @ W
probs = f.softmax(logits, dim = 1)
loss = -probs[torch.arange(5), ys].log().mean()

In [ ]:
# backward pass
W.grad = None

In [ ]:
print(loss)
loss.backward()
lr = 0.01
W.data -= lr * W.grad

In [ ]:
logits = xs_encoded @ W
probs = f.softmax(logits, dim = 1)
loss = -probs[torch.arange(5), ys].log().mean()
print(loss)

In [ ]:
# Lmao build the real dataset
xs, ys = [], []
for word in words[:]:
    chs = ['.'] + list(word) + ['.']
    for c1, c2 in zip(chs, chs[1:]):
        ix1 = stoi[c1]
        ix2 = stoi[c2]
        xs.append(ix1)
        ys.append(ix2)
xs = torch.tensor(xs)
ys = torch.tensor(ys)

In [ ]:
print(xs.nelement())
print(ys.nelement())

In [ ]:
xs_encoded = f.one_hot(xs, num_classes=27).float()
xs_encoded.shape

In [ ]:
epochs = 100
lr = 100
W = torch.randn((27, 27), requires_grad=True)
losses = []
for k in range(epochs):
    logits = (xs_encoded @ W)
    probs = f.softmax(logits, dim=1)
    loss = -probs[torch.arange(xs.nelement()), ys].log().mean()
    W.grad = None
    loss.backward()
    W.data -= lr * W.grad
    losses.append(loss.item())

print(losses[-1])
plt.plot(range(epochs), losses)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss vs Epochs')
plt.show()

In [ ]:
def predict(ix: int) -> int:
    ix_encoded = f.one_hot(torch.tensor(ix), num_classes=27).float()
    logits = ix_encoded @ W
    probs = f.softmax(logits, dim = 0)
    return torch.argmax(probs).item()
print(predict(0))

In [ ]:
print(stoi['.'])

In [ ]:
ix = 26
output = ""
stop = 0
while True:
  ix = predict(ix)
  print(itos[ix])
  output += itos[ix]
print(output)